In [1]:
import os
import requests
from aisetup import print_llm_response
from dotenv import load_dotenv

In [2]:
%pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 168.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 160.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install ipywidgets pdfplumber


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pdfplumber PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pdfplumber

In [6]:
# Get the OpenAI API key from the .env file
load_dotenv('.env', override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = openai_api_key)




NameError: name 'OpenAI' is not defined

In [ ]:
def get_llm_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant.", 
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0.0, # change this to a value between 0 and 2
    )
    response = completion.choices[0].message.content
    return response

In [ ]:
def extract_text_from_pdf(file_path: str) -> str:
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

In [ ]:
def pdf_to_dynamic_json(file_path: str):
    # Step 1: Extract text
    raw_text = extract_text_from_pdf(file_path)

    # Step 2: Define prompt
    prompt = f"""
    You are a document parser.
    Task: Convert the following text into structured JSON.

    Rules:
    - Detect fields automatically (keys can vary by document).
    - Use field names as keys, and their values as JSON values.
    - Always output valid JSON only (no explanation, no text outside JSON).
    - If you are not sure about a value, set it as null.

    Text:
    {raw_text}
    """

    # Step 3: Call LLM
    response = get_llm_response(prompt)

    return response

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import io

upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload_widget)

def save_uploaded_file(upload_widget):
    if upload_widget.value:
        # Get the first uploaded file
        file_info = list(upload_widget.value.values())[0]
        file_name = file_info['metadata']['name']
        content = file_info['content']
        # Save to disk
        with open(file_name, "wb") as f:
            f.write(content)
        return file_name
    return None


In [ ]:

def save_uploaded_file(upload_widget):
    if upload_widget.value:
        # In new ipywidgets, value is a tuple of dicts
        file_info = upload_widget.value[0]
        file_name = file_info['name']
        content = file_info['content']
        # Save to disk
        with open(file_name, "wb") as f:
            f.write(content)
        return file_name
    return None


In [ ]:
file_path = save_uploaded_file(upload_widget)
print("Saved file:", file_path)

In [ ]:
json_data = pdf_to_dynamic_json(file_path)
print(json_data)
